In [ ]:
import os 
from skimage import io
from skimage import exposure

import gzip
import gc
import pprint as pp
import numpy as np
import pickle
import matplotlib.pyplot as plt
import logging
import random
from datetime import datetime

In [ ]:
kernel_size = 16
half_kernel = int(kernel_size/2)

In [ ]:
log_location = "../logs/"
PATH_TO_VALIDATION = "../data/validation"

In [ ]:
logger = logging.getLogger()
logging.basicConfig(format="%(asctime)-15s %(message)s",
                    level=logging.DEBUG,
                    filename=os.path.join(log_location,'keras.' + datetime.now().strftime("%Y%m%d%H%M%S.%f") + '.log'))

In [ ]:
images_to_process = {}

for current_file in os.listdir(os.path.join(PATH_TO_VALIDATION,"original")):
    current_name = current_file.split(".")[0]
    images_to_process[current_name] = { "name" : current_name, "original": current_file }
    
for current_file in os.listdir(os.path.join(PATH_TO_VALIDATION,"masks")):
    current_name = current_file.split(".")[0]
    images_to_process[current_name]["mask"] = current_file 

for current_file in os.listdir(os.path.join(PATH_TO_VALIDATION)):
    if not current_file.startswith("0."):
        continue
    for current_file_validation in os.listdir(os.path.join(PATH_TO_VALIDATION,current_file)):
        current_name = current_file_validation.split(".")[0]
        images_to_process[current_name][current_file] = current_file_validation 

In [ ]:
images_keys = list(images_to_process.keys())
images_attributes = list(images_to_process[images_keys[0]].keys())
images_precision = [x for x in images_attributes if x.startswith('0.')]

In [ ]:
for current_image in images_keys:
    mask_file = os.path.join(PATH_TO_VALIDATION,"masks",images_to_process[current_image]['mask'])
    image_mask =  io.imread(mask_file)
    images_to_process[current_image]['marked_pixels'] = str(image_mask.sum() / 255)
    for current_precision in images_precision:
        mask_file = os.path.join(PATH_TO_VALIDATION,'and_mask',images_to_process[current_image]['name'] + '.mask.png')
        io.imsave(mask_file,image_mask)
       
        current_precision_file = os.path.join(PATH_TO_VALIDATION,current_precision,images_to_process[current_image][current_precision])
        precision_mask =  io.imread(current_precision_file)
        precision_mask = np.pad(precision_mask,((half_kernel,half_kernel), (half_kernel,half_kernel)),'constant')
        precision_mask = precision_mask.astype('uint8')

        #print(precision_mask.dtype)
        #print(np.unique(precision_mask))
        #print(image_mask.dtype)
        #print(np.unique(image_mask))
        #break
        
        generated_file = os.path.join(PATH_TO_VALIDATION,'and_mask',images_to_process[current_image]['name'] + current_precision +'.png')
        io.imsave(generated_file,precision_mask)
        
        current_correct_file = os.path.join(PATH_TO_VALIDATION,'and_mask',images_to_process[current_image]['name'] + current_precision +'.correct.png')
        correct_pixels = np.bitwise_and(image_mask,precision_mask )
        images_to_process[current_image][current_precision + '.correct'] = str(correct_pixels.sum() / 255)
        io.imsave(current_correct_file,correct_pixels)

        current_incorrect_file = os.path.join(PATH_TO_VALIDATION,'and_mask',images_to_process[current_image]['name'] + current_precision +'.incorrect.png')
        incorrect_pixels = np.bitwise_xor(image_mask,precision_mask )
        images_to_process[current_image][current_precision + '.incorrect'] = str(incorrect_pixels.sum() / 255)
        io.imsave(current_incorrect_file,incorrect_pixels)        

In [ ]:
csv_consolidation = os.path.join(PATH_TO_VALIDATION,'and_mask','results.csv')
with open(csv_consolidation,'w') as f:
    for current_image in images_keys:
        lista_campos_imagem = [images_to_process[current_image]['name'],images_to_process[current_image]['marked_pixels']]
        for current_precision in images_precision:
            lista_campos_imagem.append(images_to_process[current_image][current_precision + '.incorrect'])
            lista_campos_imagem.append(images_to_process[current_image][current_precision + '.correct'])
        f.write(",".join(lista_campos_imagem) + "\n")
    